## Ejercicio III


0. Crea un nuevo notebook de jupyter, guárdelo en la carpeta notebooks con el nombre `3_nb_***.ipynb` donde `***` es el github del integrante 1. Una vez que tenga todo el ejercicio listo, limpie los outputs, reinicie el kernel y corra todo el notebook. Luego, exporta el notebook como html y guárdalo en la carpeta `output`.

1. En el notebook `3_nb_***.ipynb`, cargue los dos archivos `d2019.csv` y `d2020.txt` en dos dataframes llamados `df2019` y `df2020` respectivamente. Luego, usando el método `melt`, genera un dataframe llamado `df_sii` donde estén incluidos los datos del 2019 y del 2020, que tenga las columnas `year`,`mes`, `dia`, `valor`, y guardalo como csv en la carpeta `data/interim` con el nombre `df_sii.csv`.

In [1]:
import pandas as pd

In [4]:
df2019 = pd.read_csv('../data/interim/d2019.csv')
df2019 = df2019.iloc[:,1:]
df2020 = pd.read_csv('../data/interim/d2020.txt',sep=' ')
df2020 = df2020.iloc[:,1:]
df_sii = pd.concat([df2019,df2020])

In [5]:
mes = df2019.columns.to_list()[1:-1]

In [6]:
df_sii = pd.melt(df_sii, id_vars=['year','Día'], value_vars=mes, var_name='mes')
df_sii.mes = df_sii.mes.map(lambda x: mes.index(x)+1 )
## Otra forma ver abajo
## Mapeo de meses -> https://interactivechaos.com/es/manual/tutorial-de-pandas/uso-de-un-diccionario-como-funcion-de-mapeo
df_sii = df_sii.sort_values(by=['year','mes','Día'])
df_sii.to_csv('../data/interim/df_sii.csv')

In [7]:
df_sii

,year,Día,mes,value
0,2019,1,1,0.00
1,2019,2,1,694.77
2,2019,3,1,697.09
3,2019,4,1,697.64
4,2019,5,1,0.00
...,...,...,...,...
739,2020,27,12,0.00
740,2020,28,12,710.26
741,2020,29,12,710.64
742,2020,30,12,711.24


2. Usando el método `groupby` en `df_sii`:
    * Agrupa por `mes`, con el argumento `as_index = True` y usando el método `aggregation` con el método `mean`, imprime el resultado.
    * Agrupa por `[year, mes]`, con el argumento `as_index = False` y usando el método `agg` con el método `mean`, imprime el resultado
    * Compara ambos resultados. ¿Qué cambia al usar `as_index = False`? ¿Al agrupar por `mes`, con respecto a `[year,mes]`?

In [11]:
df_sii.year = df_sii.year.astype('category')
df_sii['Día'] = df_sii['Día'].astype('category')
df_sii['mes'] = df_sii['mes'].astype('category')
# df_sii['Día'] = df_sii['Día'].astype('category')

In [12]:
df_sii.groupby(by='mes', as_index=True).agg('mean')

,value
mes,
1,514.413065
2,468.607742
3,523.992097
4,515.102258
5,486.231935
6,492.197742
7,521.892742
8,507.510806
9,470.538871


In [13]:
df_sii.groupby(by=['year','mes'], as_index=True).agg('mean')

value
year mes            
2019 1    480.495484
     2    423.422258
     3    452.298387
     4    452.109032
     5    468.776774
     6    446.715484
     7    486.880968
     8    483.476452
     9    417.160000
     10   511.700323
     11   500.987097
     12   497.026129
2020 1    548.330645
     2    513.793226
     3    595.685806
     4    578.095484
     5    503.687097
     6    537.680000
     7    556.904516
     8    531.545161
     9    523.917742
     10   533.987419
     11   516.792903
     12   474.021290

3. ¿Qué podrías inferir de los *missing values* y los días hábiles? Úsalos para crear una nueva variable llamada `dia_de_la_semana` en `df_sii` con valores "Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo".

In [91]:
df_sii.columns
df_sii['fecha'] = df_sii.apply(lambda x: str(x['year'])+'-'+str(x['mes'])+'-'+str(x['Día']), axis=1)
df_sii['fecha']= pd.to_datetime(df_sii.fecha, errors='coerce')
df_sii['day'] = df_sii.fecha.dt.day_name()

In [92]:
df_sii[df_sii.value==0].day.value_counts()

Saturday     104
Sunday       104
Friday         7
Thursday       6
Tuesday        5
Wednesday      4
Monday         2
Name: day, dtype: int64

In [93]:
# Dias que no pudo realizar la conversión debido a la inexactitud de los datos
df_sii[df_sii.day.isnull()]


,year,Día,mes,value,fecha,day
90,2019,29,2,0.0,NaT,NaN
91,2019,30,2,0.0,NaT,NaN
92,2019,31,2,0.0,NaT,NaN
216,2019,31,4,0.0,NaT,NaN
340,2019,31,6,0.0,NaT,NaN
526,2019,31,9,0.0,NaT,NaN
650,2019,31,11,0.0,NaT,NaN
122,2020,30,2,0.0,NaT,NaN
123,2020,31,2,0.0,NaT,NaN
247,2020,31,4,0.0,NaT,NaN


4. Explora el archivo `dolar_observado_bc.xlsx` y su estructura. 
    * Lee cada hoja por separado con `read_excel`, usando los argumentos `skiprows=2, dtype={'Periodo':str }`. 
    * Ocupa el método `str.split` para crear las columnas `dia`,`mes` y `year` en cada dataframe.
    * Crea un dataframe que una todos los dataframes en uno solo con las mismas columnas, y nombralo `df_bc_all`.
    * Crea una carpeta dentro de la carpeta `data/processed` llamada `dolar_bc_year`, agrupa por `year` usando `groupby` y guarda un CSV para cada año desde 1982 hasta 2022, con los nombres `dolar_bc_YYYY.csv` donde `YYYY` es el año. Busca una manera de hacerlo iterando y no uno a uno.
    * Crea un dataframe que tenga los años 2019 y 2020 que se llame `df_bc_2019_2020`
    * Compara los valores de `df_bc_2019_2020` y `df_sii`. ¿Son iguales o diferentes? 

In [107]:
df_dolar_observado = pd.read_excel('../data/raw/dolar_observado_bc.xlsx',
    skiprows=2,
    dtype={'Periodo':str}
)


In [108]:
df_dolar_observado[['year','mes','dia']]=df_dolar_observado.Periodo.str.split(pat='-', expand=True)
df_dolar_observado['dia'] = df_dolar_observado.dia.str.split(pat=' ',expand=True)[0]

In [110]:
df_dolar_observado.rename(columns={
    'Periodo': 'fecha',
    '1.Dólar observado': 'value'

}, inplace=True)
df_dolar_observado.fecha = pd.to_datetime(df_dolar_observado.fecha, format='%Y-%m-%d', errors='coerce')
df_dolar_observado

,fecha,value,year,mes,dia
0,1982-08-09,55.65,1982,08,09
1,1982-08-10,62.79,1982,08,10
2,1982-08-11,64.07,1982,08,11
3,1982-08-12,60.52,1982,08,12
4,1982-08-13,56.87,1982,08,13
...,...,...,...,...,...
2697,1989-12-27,296.62,1989,12,27
2698,1989-12-28,296.69,1989,12,28
2699,1989-12-29,296.58,1989,12,29
2700,1989-12-30,NaN,1989,12,30


In [171]:
# Ajustes para hacer la union de los dos dataframes
df_sii['dia']= df_sii['Día']
df_sii_para_union = df_sii[['fecha','value','year','mes','dia']]

In [112]:
df_bc_all = pd.concat([df_sii_para_union, df_dolar_observado])
df_bc_all

,fecha,value,year,mes,dia
0,2019-01-01,0.00,2019,1,1
1,2019-01-02,694.77,2019,1,2
2,2019-01-03,697.09,2019,1,3
3,2019-01-04,697.64,2019,1,4
4,2019-01-05,0.00,2019,1,5
...,...,...,...,...,...
2697,1989-12-27,296.62,1989,12,27
2698,1989-12-28,296.69,1989,12,28
2699,1989-12-29,296.58,1989,12,29
2700,1989-12-30,NaN,1989,12,30


In [113]:
import os
os.mkdir('../data/processed/dolar_bc_year')

In [128]:
df_bc_all[df_bc_all.year=='1982']

,fecha,value,year,mes,dia
0,1982-08-09,55.65,1982,08,09
1,1982-08-10,62.79,1982,08,10
2,1982-08-11,64.07,1982,08,11
3,1982-08-12,60.52,1982,08,12
4,1982-08-13,56.87,1982,08,13
...,...,...,...,...,...
140,1982-12-27,73.25,1982,12,27
141,1982-12-28,73.52,1982,12,28
142,1982-12-29,73.55,1982,12,29
143,1982-12-30,73.57,1982,12,30


In [135]:
def export_csv(anio, df):
    path_=f'../data/processed/dolar_bc_year/dolar_bc_{anio}.csv'
    df.to_csv(path_, index=False)

for i in range (1982,2023):
    export_csv(i,df_bc_all[df_bc_all.year==str(i)])
    

In [145]:
df_bc_all

,fecha,value,year,mes,dia
0,2019-01-01,0.00,2019,1,1
1,2019-01-02,694.77,2019,1,2
2,2019-01-03,697.09,2019,1,3
3,2019-01-04,697.64,2019,1,4
4,2019-01-05,0.00,2019,1,5
...,...,...,...,...,...
2697,1989-12-27,296.62,1989,12,27
2698,1989-12-28,296.69,1989,12,28
2699,1989-12-29,296.58,1989,12,29
2700,1989-12-30,NaN,1989,12,30


In [147]:
df_bc_2019_2020 = df_bc_all.query("year==2019 | year==2020")
df_bc_2019_2020

,fecha,value,year,mes,dia
0,2019-01-01,0.00,2019,1,1
1,2019-01-02,694.77,2019,1,2
2,2019-01-03,697.09,2019,1,3
3,2019-01-04,697.64,2019,1,4
4,2019-01-05,0.00,2019,1,5
...,...,...,...,...,...
739,2020-12-27,0.00,2020,12,27
740,2020-12-28,710.26,2020,12,28
741,2020-12-29,710.64,2020,12,29
742,2020-12-30,711.24,2020,12,30


In [172]:
df_sii_para_union['fecha'] = df_sii_para_union['fecha'].dt.strftime('%Y-%m-%d')


/tmp/ipykernel_271423/535718502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sii_para_union['fecha'] = df_sii_para_union['fecha'].dt.strftime('%Y-%m-%d')


In [175]:
df_bc_2019_2020

,fecha,value,year,mes,dia
0,2019-01-01,0.00,2019,1,1
1,2019-01-02,694.77,2019,1,2
2,2019-01-03,697.09,2019,1,3
3,2019-01-04,697.64,2019,1,4
4,2019-01-05,0.00,2019,1,5
...,...,...,...,...,...
739,2020-12-27,0.00,2020,12,27
740,2020-12-28,710.26,2020,12,28
741,2020-12-29,710.64,2020,12,29
742,2020-12-30,711.24,2020,12,30


In [177]:
dif = df_bc_2019_2020.value - df_sii_para_union.value

In [180]:
dif.value_counts()

0.0    744
Name: value, dtype: int64

5. Haz un `merge` de la columna `dia_de_la_semana` del `df_sii` al dataframe `df_bc_all`. Luego, extiende la variable `dia_de_la_semana` para todos los valores. Finalmente, crea una variable con las combinaciones `[dia,mes]` que tienen al menos un `NaN` que sea en día de semana (es decir, excluyendo Sábado y Domingo). Luego, cuenta cuantos `NaN`(incluyendo Sábados y Domingos) hay para cada combinación`[dia/mes]`. ¿Qué puedes inferir de los días con mayor cantidad de `NaN`?  


In [242]:
df_bc_all.shape

(3446, 5)

In [243]:
df_sii.shape

(744, 7)

In [250]:
df_merge = df_bc_all.merge(
    df_sii[['fecha','day']],
    on='fecha',
    how='left',
)

In [251]:
df_merge['day'] = df_merge.fecha.dt.day_name()

In [253]:
df_merge.mes=df_merge.mes.map(lambda x: str(0)+str(x) if (int(x)<9 and len(str(x)) < 2) else str(x))
df_merge['dia/mes']=df_merge.apply(lambda x:
    str(x.dia)+'/'+str(x.mes),
    axis=1
)

In [254]:
df_merge

,fecha,value,year,mes,dia,day,dia/mes
0,2019-01-01,0.00,2019,01,1,Tuesday,1/01
1,2019-01-02,694.77,2019,01,2,Wednesday,2/01
2,2019-01-03,697.09,2019,01,3,Thursday,3/01
3,2019-01-04,697.64,2019,01,4,Friday,4/01
4,2019-01-05,0.00,2019,01,5,Saturday,5/01
...,...,...,...,...,...,...,...
3597,1989-12-27,296.62,1989,12,27,Wednesday,27/12
3598,1989-12-28,296.69,1989,12,28,Thursday,28/12
3599,1989-12-29,296.58,1989,12,29,Friday,29/12
3600,1989-12-30,NaN,1989,12,30,Saturday,30/12


In [255]:
df_merge[df_merge['value']==0][['dia/mes','value']].value_counts()

dia/mes  value
31/02    0.0      26
31/9     0.0      26
31/11    0.0      26
31/06    0.0      26
31/04    0.0      26
                  ..
2/08     0.0       1
2/11     0.0       1
20/01    0.0       1
20/04    0.0       1
9/11     0.0       1
Length: 214, dtype: int64

6. Calcula el promedio del precio del dolar en los días de cumpleaños del grupo para cada año entre 1983 y 2021 y el promedio del precio del dolar para cada año completo entre 1983 y 2021, y muestra una tabla donde se vean ambos resultados. ¿Qué diferencias hay en ambos indicadores en cada año?

In [260]:
integrantes =['15/03','04/08','01/07','02/06']
anios_query = [str(x) for x in range(1983,2022)]
promedios = []
for i in integrantes:
    promedios.append(df_merge[(df_merge['dia/mes']==i) & (df_merge.year.isin(anios_query))].value.mean())

promedios_df = pd.DataFrame({
    'integrantes': ['i1','i2','i3','i4'],
    'fecha_nacimiento': integrantes,
    'promedio': promedios
})
promedios_df
    


,integrantes,fecha_nacimiento,promedio
0,i1,15/03,159.2300
1,i2,04/08,203.3140
2,i3,01/07,179.0940
3,i4,02/06,183.4625
